In [9]:
import os
import json

with open(os.path.join("model", "whisper", "train_packet.json")) as f:
    train_packet_loss = json.load(f)["train_loss"]
with open(os.path.join("model", "whisper", "train_flow.json")) as f:
    train_flow_loss = json.load(f)["train_loss"]

benign_traffic_list = ["benign2"]
attack_traffic_list = ["BruteForce-Web", "BruteForce-XSS", "mirai", "infiltration", "SQL_Injection", "osscan", "ssldosA10only"]

granularity_list = ["packet", "flow"]
train_loss_list = [train_packet_loss, train_flow_loss]

In [10]:
phi = 4

for benign_traffic in benign_traffic_list:
    filename = os.path.join("result", "whisper", benign_traffic)
    for idx in range(2):
        granularity = granularity_list[idx]
        train_loss = train_loss_list[idx]
        with open(f"{filename}-{granularity}.json", "r") as f:
            result = json.load(f)
            bad_count = 0
            for dict_pair in result:
                if dict_pair["loss"] > phi * train_loss:
                    bad_count += 1
            print(f"{benign_traffic}-{granularity} accuracy: {1 - bad_count / len(result)}")

for attack_traffic in attack_traffic_list:
    filename = os.path.join("result", "whisper", attack_traffic)
    for idx in range(2):
        granularity = granularity_list[idx]
        train_loss = train_loss_list[idx]
        with open(f"{filename}-{granularity}.json", "r") as f:
            result = json.load(f)
            bad_count = 0
            for dict_pair in result:
                if dict_pair["loss"] <= phi * train_loss:
                    bad_count += 1
            if len(result) > 1:
                print(f"{attack_traffic}-{granularity} accuracy: {1 - bad_count / len(result)}")
            else:
                print(f"{attack_traffic}-{granularity} accuracy: empty result")

benign2-packet accuracy: 0.5897380385364798
benign2-flow accuracy: 0.9722772277227723
BruteForce-Web-packet accuracy: 1.0
BruteForce-Web-flow accuracy: 1.0
BruteForce-XSS-packet accuracy: 1.0
BruteForce-XSS-flow accuracy: 1.0
mirai-packet accuracy: 0.3630716181823157
mirai-flow accuracy: 0.6956521739130435
infiltration-packet accuracy: empty result
infiltration-flow accuracy: 0.8333333333333334
SQL_Injection-packet accuracy: 1.0
SQL_Injection-flow accuracy: empty result
osscan-packet accuracy: 0.019061583577712593
osscan-flow accuracy: empty result
ssldosA10only-packet accuracy: 1.0
ssldosA10only-flow accuracy: empty result


In [10]:
import pandas as pd
import os
import json

In [12]:
benign_df = pd.read_csv("train_set/benign1.csv")
num_benign = benign_df.shape[0]

attack_filenames = [os.path.join("attack_set", x) for x in 
                    os.listdir("attack_set") if x.endswith(".csv")]
with open("attacker-ips.json", "r") as f:
    attack_ips_dict = json.load(f)

for attack_filename in attack_filenames:
    attack_df = pd.read_csv(attack_filename)
    num_attack = 0
    file_key = os.path.basename(attack_filename).split(".")[0]
    cur_attack_ips = attack_ips_dict.get(file_key, [])
    attack_packet_count = 0
    for row in attack_df.iterrows():
        if row[1]["src_ip"] in cur_attack_ips:
            num_attack += 1
    print(f"{attack_filename}: num= {num_attack}, ratio= {100 * num_attack / num_benign:.2f}%")

attack_set/LDoS_small.csv: num= 97750, ratio= 51.21%
attack_set/osscan.csv: num= 2337, ratio= 1.22%
attack_set/infiltration.csv: num= 2125, ratio= 1.11%
attack_set/HOIC_small.csv: num= 45852, ratio= 24.02%
attack_set/BruteForce-Web.csv: num= 10864, ratio= 5.69%
attack_set/LOIC_UDP_small.csv: num= 99992, ratio= 52.38%
attack_set/SQL_Injection.csv: num= 103, ratio= 0.05%
attack_set/ssldosA.csv: num= 100987, ratio= 52.91%
attack_set/fuzzscan.csv: num= 4197, ratio= 2.20%
attack_set/BruteForce-XSS.csv: num= 7727, ratio= 4.05%


In [ ]:
with open("cic-attacker-ips.json", "r") as f:
    attack_ips_dict = json.load(f)

attack_filenames = [os.path.join("cic-ids", x) for x in 
                    os.listdir("cic-ids") if x.endswith(".csv")]
for filename in attack_filenames:
    attack_df = pd.read_csv(filename)
    file_key = os.path.basename(filename).split(".")[0]
    cur_attack_ips = attack_ips_dict.get(file_key, [])
    attack_packet_count = 0
    for row in attack_df.iterrows():
        if row[1]["src_ip"] in cur_attack_ips or row[1]["dst_ip"] in cur_attack_ips:
            attack_packet_count += 1
    print(f"{filename}: attack= {attack_packet_count}, ratio={100*attack_packet_count/attack_df.shape[0]:.2f}%")